# VND Banknotes Classifer

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import imageio
import PIL
%matplotlib inline

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.callbacks import ModelCheckpoint

## Working with Image directory

In [ ]:
# Unzip
!unzip -q /content/gdrive/MyDrive/'FTMLE | 2021.07 | ML30'/Week_8/DATASET/VND_Banknotes_dataset.zip -d /content/gdrive/MyDrive/Project_ML30/vnd_banknotes_classifier/

In [ ]:
import pathlib
data_dir = pathlib.Path('/content/gdrive/MyDrive/Project_ML30/vnd_banknotes_classifier')

In [ ]:
# Save result on google drive
PROJECT = data_dir
RESULT = PROJECT/'Results'
SAVED_MODEL = RESULT/'Saved_model'
IMG_SIZE = 224 
BATCH_SIZE = 32

## Prepare ImageDataGenerator

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def generator_augmentation_maker():
  train_gen = ImageDataGenerator(
                                rotation_range=40,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                fill_mode='nearest',
                                validation_split=0.2)     
  
  val_gen = ImageDataGenerator(validation_split=0.2)  

  train_generator = train_gen.flow_from_directory(
                                data_dir,  
                                target_size=(IMG_SIZE, IMG_SIZE),  
                                batch_size=BATCH_SIZE,
                                class_mode='sparse',
                                shuffle=True,
                                seed=42,           
                                subset='training')  

  validation_generator = val_gen.flow_from_directory(
                                data_dir,
                                target_size=(IMG_SIZE, IMG_SIZE),
                                batch_size=BATCH_SIZE,
                                class_mode='sparse',
                                shuffle=False,       
                                seed=42,              
                                subset='validation')  
  return train_generator, validation_generator

In [ ]:
train_generator, validation_generator = generator_augmentation_maker()

## Build Model

In [ ]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
base_model = keras.applications.MobileNetV2(weights='imagenet',
                                   input_shape=(IMG_SIZE, IMG_SIZE, 3),
                                   include_top=False)
# Freeze the base model's weights
base_model.trainable=False

In [ ]:
from keras.applications.mobilenet_v2 import preprocess_input

def my_model():
  inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE,3))
  x = preprocess_input(inputs)
  
  x = base_model(x,training=False)

  x = layers.Dense(256)(x)
  x = layers.Dense(512)(x)
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.Dense(512)(x)
  x = layers.Dropout(0.2)(x)
  outputs = layers.Dense(9, activation='softmax')(x)

  model = keras.Model(inputs, outputs)

  return model

In [ ]:
model = my_model()
model.summary()

In [ ]:
model.compile(optimizer= 'Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import h5py

In [ ]:
ear_stop = EarlyStopping(monitor='val_loss', patience=9, restore_best_weights=True)
mod_chk = ModelCheckpoint(filepath='/content/gdrive/MyDrive/Project_ML30/vnd_banknotes_classifier/my_model1.h5', monitor='val_loss', save_best_only=True)
lr_rate = ReduceLROnPlateau(monitor='val_loss', patience=9, factor=0.1)


In [ ]:
hist = model.fit(train_generator,
                    validation_data=validation_generator,
                    epochs=20,
                    callbacks = [ear_stop, mod_chk, lr_rate], verbose=2)


In [ ]:
# fine tune
from keras.applications.mobilenet_v2 import preprocess_input

def my_model():
  inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE,3))
  x = preprocess_input(inputs)
  
  x = base_model(x,training=False)
  x = layers.Dense(128)(x)
  x = layers.Dense(256)(x)
  x = layers.Dense(512)(x)
  x = layers.GlobalAveragePooling2D()(x)
  x = layers.Dense(512)(x)
  x = layers.Dropout(0.2)(x)
  outputs = layers.Dense(9, activation='softmax')(x)

  model = keras.Model(inputs, outputs)

  return model

In [ ]:
model1 = my_model()
model1.summary()

In [ ]:
model1.compile(optimizer= 'Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
hist_1 = model1.fit(train_generator,
                    validation_data=validation_generator,
                    epochs=20,
                    callbacks = [ear_stop, mod_chk, lr_rate], verbose=2)

In [ ]:
# Plotting result
plt.plot(hist.history['loss'], color='blue', label='train')
plt.plot(hist.history['val_loss'], color='orange', label='train')
plt.grid(True)
plt.title("Train & test loss with eporchs\n", fontsize=16)
plt.xlabel('Train epochs', fontsize=12)
plt.ylabel('Train & test loss', fontsize=12)
plt.show()

In [ ]:
# Plotting results with accuracy
plt.plot(hist.history['accuracy'], color='blue', label='train')
plt.plot(hist.history['val_accuracy'], color='orange', label='train')
plt.grid(True)
plt.title("Train & test accuracy with eporchs\n", fontsize=16)
plt.xlabel('Train epochs', fontsize=12)
plt.ylabel('Train & test accuracy', fontsize=12)
plt.show()

In [ ]:
acc = model.evaluate(train_generator, steps=len(validation_generator), verbose = 2)

In [ ]:
# Save model
model.save('/content/gdrive/MyDrive/Project_ML30/vnd_banknotes_classifier/my_model1.h5') 

# Preadict a real image

In [ ]:
from tensorflow.keras.preprocessing import image
import matplotlib.image as mpimg

def predict(model, image_path, img_size):
  img = image.load_img(image_path, target_size=(img_size, img_size))
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis = 0)
  prediction = model.predict(img)

  for key, value in validation_generator.class_indices.items():
    if value == prediction[0].argmax():
      pred = key

  plt.figure(figsize=(9, 6))
  img = mpimg.imread(image_path)
  imgplot = plt.imshow(img)
  plt.title('Prediction: ' + pred.upper())
  plt.grid(b=None)
  plt.axis('off')
  plt.show()

  return True

In [ ]:
# up load file from google drive
from google.colab import files
uploaded = files.upload()

In [ ]:
# Predict your file
image_path= '/content/5.jpg'
predict(model, image_path, IMG_SIZE)

In [ ]:
# Predict your file
image_path= '/content/100.jpg'
predict(model, image_path, IMG_SIZE)

In [ ]:
# Predict your file
image_path= '/content/1.jpg'
predict(model, image_path, IMG_SIZE)

In [ ]:
# Predict your file
image_path= '/content/2.jpg'
predict(model, image_path, IMG_SIZE)

In [ ]:
# Predict your file
image_path= '/content/3.jpg'
predict(model, image_path, IMG_SIZE)

In [ ]:
# Predict your file
image_path= '/content/4.jpg'
predict(model, image_path, IMG_SIZE)

In [ ]:
# Predict your file
image_path= '/content/5_1.jpg'
predict(model, image_path, IMG_SIZE)